In [1]:
import pandas as pd

/tmp/ipykernel_8033/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_csv(csv_path):
    data_df = pd.read_csv(csv_path)
    return data_df

In [3]:
df = load_csv("../listings_images.csv")
df

,listingImageId,listingId,suburbName,state,suburbId,imageLink
0,158649238,15385009,GRIFFITH,ACT,1996,pics/009/15385009-1-F04DAED.jpg
1,158649239,15385009,GRIFFITH,ACT,1996,pics/009/15385009-2-981E301.jpg
2,158649240,15385009,GRIFFITH,ACT,1996,pics/009/15385009-3-37ED883.jpg
3,158649241,15385009,GRIFFITH,ACT,1996,pics/009/15385009-4-DB8E63C.jpg
4,158649242,15385009,GRIFFITH,ACT,1996,pics/009/15385009-5-B8CDE9A.jpg
...,...,...,...,...,...,...
16484,158499471,15375609,SANCTUARY COVE,QLD,17223,pics/609/15375609-14-A843F8F.jpg
16485,158499472,15375609,SANCTUARY COVE,QLD,17223,pics/609/15375609-15-0D7F791.png
16486,158499473,15375609,SANCTUARY COVE,QLD,17223,pics/609/15375609-16-9303D50.png
16487,158499474,15375609,SANCTUARY COVE,QLD,17223,pics/609/15375609-17-880316C.png


In [4]:
def create_link(link):
    base_link = "https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/"
    edited_link = link.split("/")
    return base_link + edited_link[2]

In [5]:
def load_raw_data(df):
    data_dict = []

    for index, row in df.iterrows():
        row_dict = row.to_dict()

        edited_link = create_link(row_dict["imageLink"])
        row_dict["imageLink"] = edited_link
        data_dict.append(row_dict)

    return data_dict


In [6]:
data_dict = load_raw_data(df)

In [7]:
data_dict[:5]

[{'listingImageId': 158649238,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-1-F04DAED.jpg'},
 {'listingImageId': 158649239,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-2-981E301.jpg'},
 {'listingImageId': 158649240,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-3-37ED883.jpg'},
 {'listingImageId': 158649241,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-4-DB8E63C.jpg'},
 {'listingImageId': 158649242,
  'listingId': 15385009,
  'suburbName': 'GRIFFIT

In [8]:
import requests as req
import io
from PIL import Image
import numpy as np
import torch

In [9]:
def download_image(image_link):

    try:
        image_content = req.get(image_link, timeout = 10).content
        image_file = io.BytesIO(image_content)
        pil_image = Image.open(image_file)
        print("downloaded")
    except Exception as e:
        print("An error occurred while downloading the image!", e)

    return pil_image


In [10]:
model = torch.hub.load('../models/yolov5','custom', path="../models/yolov5/runs/train/exp5/weights/best.pt", source="local")

YOLOv5 🚀 v7.0-216-gec690e4 Python-3.10.12 torch-2.2.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [11]:
def infer(image):
    results = model(image)
    return eval(results.pandas().xyxy[0].to_json(orient='records'))

In [18]:
def splitting(image,data):
    path = "../data/interrim"
    if len(data['prediction']) == 0:
        return 
    elif len(data['prediction']) == 1:
        try:
            category = data['prediction'][0]['name']
            image_name = f"{category}_{data['listingImageId']}.png"
            with open(f"{path}/{category}/{image_name}", 'wb') as f:
                image.save(f)
            print(f"{image_name} donwloaded at {path}/{category}")
        except Exception as e:
            print(f"error while saving image : {e}")
    else:
        try:
            folder_name = "mix"
            category = "_".join([i['name'] for i in data['prediction']])
            image_name = f"{category}_{data['listingImageId']}.png"
            with open(f"{path}/{folder_name}/{image_name}",'wb') as f:
                image.save(f)
            print(f"{image_name} donwloaded at {path}/{folder_name}")
        except Exception as e:
            print(f"error while saving image : {e}")
            



In [19]:
def download_and_infer(data_dict):
        
    index = 0
    while index < len(data_dict):
        print(f"index : {index}")
        image_link = data_dict[index]['imageLink']
        image = download_image(image_link)
        print(image.getbands())

        if len(image.getbands()) <= 3:
            results = infer(image)
            data_dict[index]['prediction'] = results

            try:
                file_path = '../data/interrim/all'
                with open(f"{file_path}/{data_dict[index]['listingImageId']}.png", "wb") as f:
                    image.save(f)
                print(f"Downloaded: {data_dict[index]['listingImageId']}.png")
            except Exception as e:
                print(f"Error while saving the image : {e}")
            
            splitting(image,data_dict[index])
        
        else:
            print("An image with alpha channel found, hence discarding it!")
            data_dict.pop(index)
            continue
        index += 1
    
    print(data_dict)
        


In [20]:
test = [{'listingImageId': 158649238,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-1-F04DAED.jpg'},
 {'listingImageId': 158649239,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-2-981E301.jpg'},
 {'listingImageId': 158649240,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://images.samsung.com/is/image/samsung/p6pim/in/2307/gallery/in-galaxy-watch6-r935-sm-r930nzeains-thumb-537406388'},
 {'listingImageId': 158649241,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-4-DB8E63C.jpg'},
 {'listingImageId': 158649242,
  'listingId': 15385009,
  'suburbName': 'GRIFFITH',
  'state': 'ACT',
  'suburbId': 1996,
  'imageLink': 'https://resi.stgz.view.com.au/viewstatic/images/listing/acw/800-w/15385009-5-B8CDE9A.jpg'}]

In [21]:
download_and_infer(data_dict[21:30])

index : 0
downloaded
('R', 'G', 'B')
Downloaded: 158649329.png
living_room_158649329.png donwloaded at ../data/interrim/living_room
index : 1
downloaded
('R', 'G', 'B')
Downloaded: 158649330.png
balcony_living_room_158649330.png donwloaded at ../data/interrim/mix
index : 2
downloaded
('R', 'G', 'B')
Downloaded: 158649331.png
living_room_158649331.png donwloaded at ../data/interrim/living_room
index : 3
downloaded
('R', 'G', 'B')
Downloaded: 158649332.png
kitchen_refrigerator_158649332.png donwloaded at ../data/interrim/mix
index : 4
downloaded
('R', 'G', 'B')
Downloaded: 158649333.png
bedroom_158649333.png donwloaded at ../data/interrim/bedroom
index : 5
downloaded
('R', 'G', 'B')
Downloaded: 158649334.png
bedroom_158649334.png donwloaded at ../data/interrim/bedroom
index : 6
downloaded
('R', 'G', 'B')
Downloaded: 158649335.png
bedroom_living_room_158649335.png donwloaded at ../data/interrim/mix
index : 7
downloaded
('R', 'G', 'B')
Downloaded: 158649336.png
error while saving image : [

In [2]:
import os
os.mkdir("./test")


In [9]:
import requests as req
from requests.exceptions import ConnectionError
from requests.exceptions import Timeout
import time

while True:
    try:
        image_content = req.get("https://www.alleycat.org/wp-content/uploads/2019/03/FELV-cat.jpg", timeout = 10).content
        print("Downloaded!")
        time.sleep(4)
    except ConnectionError:
        print("Connection Error! retrying")
        time.sleep(4)
        continue
    except Timeout:
        print("Timeout")

Downloaded!
Downloaded!
Downloaded!
Connection Error! retrying
Connection Error! retrying
Connection Error! retrying
Downloaded!
Downloaded!


KeyboardInterrupt: 